# COVID PS and RX dataset analysis

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score
from sklearn.linear_model import LogisticRegression
from typing import Any, Dict, List, Tuple
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import time
import sys
import os

### Paths to data

In [ ]:
data_path = "/Users/manuel/Desktop/BiomedDataAnalysisCourse/project/data/"
ps_rx_fname = os.path.join(data_path, "merged_data_processed_corrected.csv")

### Define functions used throughout the analysis

In [ ]:
def compute_age(birth_year: int, visit_year: int) -> int:
    assert isinstance(birth_year, int)
    assert isinstance(visit_year, int)
    assert birth_year < visit_year
    age = visit_year - birth_year
    return age

In [ ]:
def fillna_column(column_data: pd.Series, dtype: str) -> List[Any]:
    assert isinstance(column_data, pd.Series)
    assert isinstance(dtype, str)
    if dtype == "categorical":
        most_freq = column_data.describe()["top"]
        column_data.fillna(most_freq, inplace=True)
    elif dtype == "numerical":
        mean_val = column_data.describe()["mean"]
        column_data.fillna(mean_val, inplace=True)
    else:
        raise ValueError(f"Unknown data type ({dtype})")
    return column_data

In [ ]:
def normalize_column(column_data: pd.Series) -> List[float]:
    assert isinstance(column_data, pd.Series)
    norm_data = [None] * len(column_data)
    column_data = column_data.tolist()
    for i,v in enumerate(column_data):
        norm_data[i] = (column_data[i] - min(column_data)) / (max(column_data) - min(column_data))
    assert len(norm_data) == len(column_data)
    return norm_data

In [ ]:
def standardize_column(column_data: pd.Series) -> List[float]:
    assert isinstance(column_data, pd.Series)
    std_data = [None] * len(column_data)
    column_data = column_data.tolist()
    for i,v in enumerate(column_data):
        std_data[i] = (column_data[i] - np.mean(column_data)) / np.std(column_data)
    assert len(std_data) == len(column_data)
    return std_data

In [ ]:
def remove_measures(column_data: pd.Series) -> List[float]:
    assert isinstance(column_data, pd.Series)
    corrected_data = []
    for v in column_data.tolist():
        if str(v) == "nan":
            corrected_data.append(v)
        else:
            fields = v.split("10^9")
            value = float(fields[0].replace(",", "."))
            corrected_data.append(value)
    assert len(column_data) == len(corrected_data)
    return corrected_data

## Start analysis

Load and visualize the dataset.

In [ ]:
ps_rx_df = pd.read_csv(ps_rx_fname, delimiter=";", decimal=",")
ps_rx_df.head()

Remove units of measure from some columns of the DataFrame.

In [ ]:
for col in [
    "FIELDSET_PS-BLOOD_COUNT_LEUCOCITI", "FIELDSET_PS-BLOOD_COUNT_NEUTROFILI"
]:
    ps_rx_df[col] = remove_measures(ps_rx_df[col])
ps_rx_df.head()

#### Preprocessing steps

Compute patients age at ER visit time.

In [ ]:
ps_rx_df["AGE"] = ps_rx_df.apply(lambda x : compute_age(int(x[1]), int(x[-1].split("/")[-1])), axis=1)
ps_rx_df.head()

Drop columns not used throughout the analysis.

In [ ]:
drop_cols = [
    "BIRTHDAY",
    "DEAD_DATE",
    "STOP",
    "START",
    "CODE"
]
ps_rx_df.drop(drop_cols, axis=1, inplace=True)
ps_rx_df.head()  # 769 visits and 89 variables

Set visit ID as DataFrame index.

In [ ]:
ps_rx_df.index = ps_rx_df.ID.tolist()
ps_rx_df.drop(["ID"], axis=1, inplace=True)
ps_rx_df.head()  # 88 variables

Recover training data and the response we want predict.

In [ ]:
X = ps_rx_df.drop(["DEATH"], axis=1)
Y = ps_rx_df["DEATH"]
X.head()  # training data

In [ ]:
Y.head()  # response

Encode categorical variables in the dataset. For encoding we'll use `OrdinalEncoder` function from the `sklearn` Python package.

In [ ]:
# fill NaN rows for each categorical variable using the most frequent value
cat_vars = X.select_dtypes(["object"]).columns.tolist()
for cat_var in cat_vars:
    X[cat_var] = fillna_column(X[cat_var], "categorical")
X.head()

In [ ]:
# # fill NaN rows for each numerical variable using the mean value
num_vars = list(set(X.columns.tolist()).difference(set(cat_vars)))
for num_var in num_vars:
    X[num_var] = fillna_column(X[num_var], "numerical")
X.head()

In [ ]:
# categorical variables encoding
X_cat = X[cat_vars]
enc = OrdinalEncoder()
X[cat_vars] = enc.fit_transform(X_cat)
X

Visualize our dataset using dimensionality reduction via PCA.

In [ ]:
pca = PCA(n_components=2)  # use 2 components
pcs = pca.fit_transform(X)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X.index.tolist()
)
f, ax = plt.subplots(1,1,figsize=(20,10))
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", ax=ax)
ax.set_xlabel("PC1", size=16)
ax.set_ylabel("PC2", size=16)
ax.set_title("Original data", size=18)
plt.show()

There are some strong outliers in our dataset. We should remove them before proceeding.

In [ ]:
visits_to_remove = X_pcs[ 
    (X_pcs.PC1 > 3500) | (X_pcs.PC2 > 1000) 
].index.tolist()
X.drop(visits_to_remove, axis=0, inplace=True)
# adjust Y vector accordingly
Y.drop(visits_to_remove, axis=0, inplace=True)
# recompute PCs
pcs = pca.fit_transform(X)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X.index.tolist()
)
# plot PCA
f, ax = plt.subplots(1,1,figsize=(20,10))
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", ax=ax)
ax.set_xlabel("PC1", size=16)
ax.set_ylabel("PC2", size=16)
ax.set_title("Original data", size=18)
plt.show()

In [ ]:
# standardize data
X_std = pd.DataFrame()
for col in X.columns.tolist():
    X_std[col] = standardize_column(X[col])
# compute PCs
pcs = pca.fit_transform(X_std)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X_std.index.tolist()
)
# plot PCA
f, ax = plt.subplots(1,1,figsize=(20,10))
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", ax=ax)
ax.set_xlabel("PC1", size=16)
ax.set_ylabel("PC2", size=16)
ax.set_title("Original data", size=18)
plt.show()

In [ ]:
# normalize the dataset in [0,1]
X_norm = pd.DataFrame()
for col in X.columns.tolist():
    X_norm[col] = normalize_column(X[col])
# compute PCs
pcs = pca.fit_transform(X_norm)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X_norm.index.tolist()
)
# plot PCA
f, ax = plt.subplots(1,1,figsize=(20,10))
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", ax=ax)
ax.set_xlabel("PC1", size=16)
ax.set_ylabel("PC2", size=16)
ax.set_title("Original data", size=18)
plt.show()

### Unsupervised classification

#### K-means

In [ ]:
# perform k-means clustering
kmeans = KMeans(n_clusters=2, init="random", max_iter=150, random_state=0)
labels_orig = kmeans.fit_predict(X)
labels_orig = [np.abs(l - 1) for l in labels_orig]
# visualize results 
pca = PCA(n_components=2)
pcs = pca.fit_transform(X)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X.index.tolist()
)
X_pcs["Cluster"] = labels_orig
X_pcs["Death"] = Y.tolist()
f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
palette=["#D3880E", "#3B1375"]
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Death", ax=ax1)
ax1.set_xlabel("PC1", size=16)
ax1.set_ylabel("PC2", size=16)
ax1.set_title("Original data", size=18)
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Cluster", ax=ax2)
ax2.set_xlabel("PC1", size=16)
ax2.set_ylabel("PC2", size=16)
ax2.set_title("K-means clustering", size=18)
plt.show()

In [ ]:
# perform k-means clustering
kmeans = KMeans(n_clusters=2, init="random", max_iter=150, random_state=0)
labels_norm = kmeans.fit_predict(X_norm)
#labels_norm = [np.abs(l - 1) for l in labels_norm]
# visualize results 
pca = PCA(n_components=2)
pcs = pca.fit_transform(X_norm)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X_norm.index.tolist()
)
X_pcs["Cluster"] = labels_norm
X_pcs["Death"] = Y.tolist()
f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
palette=["#D3880E", "#3B1375"]
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Death", ax=ax1)
ax1.set_xlabel("PC1", size=16)
ax1.set_ylabel("PC2", size=16)
ax1.set_title("Original data", size=18)
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Cluster", ax=ax2)
ax2.set_xlabel("PC1", size=16)
ax2.set_ylabel("PC2", size=16)
ax2.set_title("K-means clustering", size=18)
plt.show()

In [ ]:
# perform k-means clustering
kmeans = KMeans(n_clusters=2, init="random", max_iter=150, random_state=0)
labels_std = kmeans.fit_predict(X_std)
labels_std = [np.abs(l - 1) for l in labels_std]
# visualize results 
pca = PCA(n_components=2)
pcs = pca.fit_transform(X_std)
X_pcs = pd.DataFrame(
    data=pcs, columns=[f"PC{i}" for i in range(1,3)], index=X_std.index.tolist()
)
X_pcs["Cluster"] = labels_std
X_pcs["Death"] = Y.tolist()
f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
palette=["#D3880E", "#3B1375"]
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Death", ax=ax1)
ax1.set_xlabel("PC1", size=16)
ax1.set_ylabel("PC2", size=16)
ax1.set_title("Original data", size=18)
sns.scatterplot(data=X_pcs, x="PC1", y="PC2", palette=palette, hue="Cluster", ax=ax2)
ax2.set_xlabel("PC1", size=16)
ax2.set_ylabel("PC2", size=16)
ax2.set_title("K-means clustering", size=18)
plt.show()

In [ ]:
# original data
print("\nOriginal data\n")
f1 = f1_score(Y, labels_orig, average="binary")
print(f"F1-score: {f1}")

# normalized data
print("\nNormalized data\n")
f1 = f1_score(Y, labels_norm, average="binary")
print(f"F1-score: {f1}")

# standardized data
print("\nStandardized data\n")
f1 = f1_score(Y, labels_std, average="binary")
print(f"F1-score: {f1}")

#### KNN algorithm

In [ ]:
# original data
print("\nOriginal data\n")
for n in range(3, 11):
    # split dataset in training and testing sets
    X_train, X_test, Y_train, Y_test_orig = train_test_split(
        X, Y, test_size=0.25 
    )
    # use KNN
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train, Y_train)
    predictions_orig = neigh.predict(X_test)
    f1 = f1_score(predictions_orig, Y_test_orig, average="binary") 
    print(f"F1-score ({n} neighbors): {f1}")

# normalized data
print("\nNormalized data\n")
for n in range(3, 11):
    # split dataset in training and testing sets
    X_train, X_test, Y_train, Y_test_norm = train_test_split(
        X_norm, Y, test_size=0.25 
    )
    # use KNN
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train, Y_train)
    predictions_norm = neigh.predict(X_test)
    f1 = f1_score(predictions_norm, Y_test_norm, average="binary") 
    print(f"F1-score ({n} neighbors): {f1}")

# standardized data
print("\nStandardized data\n")
for n in range(3, 11):
    # split dataset in training and testing sets
    X_train, X_test, Y_train, Y_test_std = train_test_split(
        X_std, Y, test_size=0.25 
    )
    # use KNN
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X_train, Y_train)
    predictions_std = neigh.predict(X_test)
    f1 = f1_score(predictions_std, Y_test_std, average="binary") 
    print(f"F1-score ({n} neighbors): {f1}")

#### Logistic regression

In [ ]:
# original data
print("\nOriginal data\n")
X_train, X_test, Y_train, Y_test_orig = train_test_split(
        X, Y, test_size=0.25
)
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
predictions_orig = clf.predict(X_test)
f1 = f1_score(predictions_orig, Y_test_orig, average="binary")
print(f"F1-score: {f1}")
# normalized data
print("\nNormalized data\n")
X_train, X_test, Y_train, Y_test_norm = train_test_split(
        X_norm, Y, test_size=0.25
)
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
predictions_norm = clf.predict(X_test)
f1 = f1_score(predictions_norm, Y_test_norm, average="binary")
print(f"F1-score: {f1}")
# standardized data
# original data
print("\nStandardized data\n")
X_train, X_test, Y_train, Y_test_std = train_test_split(
        X_std, Y, test_size=0.25
)
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
predictions_std = clf.predict(X_test)
f1 = f1_score(predictions_std, Y_test_std, average="binary")
print(f"F1-score: {f1}")



#### Support Vector Machines